In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import chebyt
from numpy.linalg import cond, lstsq


def f1(x): return np.sin(10 * x)
def f2(x): return 1 / (1 + 25 * x**2)
def f3(x): return np.where((x >= 0) & (x <= 2), 1.0, 0.0)

def chebyshev_nodes(m, interval):
    a, b = interval
    j = np.arange(1, m + 1)
    x = np.cos((2 * j - 1) * np.pi / (2 * m))
    return 0.5 * (b - a) * x + 0.5 * (a + b)

def normalized_monomial_basis(x, n):
    X = np.vander(x, n+1, increasing=True)
    norms = np.sqrt(np.sum(X**2, axis=0))
    return X / norms


def chebyshev_basis(x, n, interval):
    a, b = interval
    x_scaled = 2 * (x - a) / (b - a) - 1
    X = np.zeros((len(x), n+1))
    for i in range(n+1):
        poly = chebyt(i)
        X[:, i] = poly(x_scaled)
    return X


def compute_metrics(y_true, y_pred):
    mse = np.mean((y_true - y_pred)**2)
    rmse = np.sqrt(mse)
    max_error = np.max(np.abs(y_true - y_pred))
    y_mean = np.mean(y_true)
    ss_tot = np.sum((y_true - y_mean)**2)
    ss_res = np.sum((y_true - y_pred)**2)
    r2 = 1 - ss_res / ss_tot if ss_tot > 0 else 1
    return rmse, max_error, r2


def fit_polynomials(x, y, n, interval, x_test, y_test):

    A_mono = normalized_monomial_basis(x, n)
    cond_mono = cond(A_mono)
    coeffs_mono = lstsq(A_mono, y, rcond=None)[0]
    y_pred_mono_train = A_mono @ coeffs_mono
    A_mono_test = normalized_monomial_basis(x_test, n)
    y_pred_mono_test = A_mono_test @ coeffs_mono
    rmse_mono, max_err_mono_train, _ = compute_metrics(y, y_pred_mono_train)
    _, max_err_mono_test, _ = compute_metrics(y_test, y_pred_mono_test)


    A_cheb = chebyshev_basis(x, n, interval)
    cond_cheb = cond(A_cheb)
    coeffs_cheb = lstsq(A_cheb, y, rcond=None)[0]
    y_pred_cheb_train = A_cheb @ coeffs_cheb
    A_cheb_test = chebyshev_basis(x_test, n, interval)
    y_pred_cheb_test = A_cheb_test @ coeffs_cheb
    rmse_cheb, max_err_cheb_train, _ = compute_metrics(y, y_pred_cheb_train)
    _, max_err_cheb_test, _ = compute_metrics(y_test, y_pred_cheb_test)

    return rmse_mono, max_err_mono_train, max_err_mono_test, cond_mono, rmse_cheb, max_err_cheb_train, max_err_cheb_test, cond_cheb


datasets = [
    {"func": f1, "interval": [-1, 1], "title": "sin_10x_on_-1_1"},
    {"func": f2, "interval": [-5, 5], "title": "1_over_1_plus_25x2_on_-5_5"},
    {"func": f3, "interval": [-2, 2], "title": "step_function_on_-2_2"}
]
m_values = [50, 200, 300]
n_values = range(1, 31)


for m in m_values:
    for idx, dataset in enumerate(datasets, 1):
        plt.figure(figsize=(15, 5))
        x_train = chebyshev_nodes(m, dataset["interval"])
        y_train = dataset["func"](x_train)
        x_test = np.linspace(dataset["interval"][0], dataset["interval"][1], 1000)
        y_test = dataset["func"](x_test)

        rmse_mono_train, max_err_mono_train, max_err_mono_test, cond_mono_list = [], [], [], []
        rmse_cheb_train, max_err_cheb_train, max_err_cheb_test, cond_cheb_list = [], [], [], []

        for n in n_values:
            rmse_mono, max_err_mono_tr, max_err_mono_te, cond_mono, rmse_cheb, max_err_cheb_tr, max_err_cheb_te, cond_cheb = fit_polynomials(x_train, y_train, n, dataset["interval"], x_test, y_test)
            rmse_mono_train.append(rmse_mono)
            max_err_mono_train.append(max_err_mono_tr)
            max_err_mono_test.append(max_err_mono_te)
            cond_mono_list.append(cond_mono)
            rmse_cheb_train.append(rmse_cheb)
            max_err_cheb_train.append(max_err_cheb_tr)
            max_err_cheb_test.append(max_err_cheb_te)
            cond_cheb_list.append(cond_cheb)

        # RMSE Train
        plt.subplot(1, 3, 1)
        plt.plot(n_values, rmse_mono_train, label="Monomial (Normalized)", marker='o')
        plt.plot(n_values, rmse_cheb_train, label="Chebyshev", marker='s')
        plt.xlabel("Polynomial Degree (n)")
        plt.ylabel("RMSE (Train)")
        plt.title(f"{dataset['title'].replace('_', ' ')}, m={m} (Train)")
        plt.legend()
        plt.grid(True)
        plt.yscale("log")

        # Max Error (Test)
        plt.subplot(1, 3, 2)
        plt.plot(n_values, max_err_mono_test, label="Monomial (Normalized)", marker='o')
        plt.plot(n_values, max_err_cheb_test, label="Chebyshev", marker='s')
        plt.xlabel("Polynomial Degree (n)")
        plt.ylabel("Max Error (Test)")
        plt.title(f"{dataset['title'].replace('_', ' ')}, m={m} (Test Max Error)")
        plt.legend()
        plt.grid(True)
        plt.yscale("log")

        # Condition Number
        plt.subplot(1, 3, 3)
        plt.plot(n_values, cond_mono_list, label="Monomial (Normalized)", marker='o')
        plt.plot(n_values, cond_cheb_list, label="Chebyshev", marker='s')
        plt.xlabel("Polynomial Degree (n)")
        plt.ylabel("Condition Number")
        plt.title(f"{dataset['title'].replace('_', ' ')}, m={m} (Condition)")
        plt.legend()
        plt.grid(True)
        plt.yscale("log")

        plt.tight_layout()
        plt.savefig(f"plot_m_{m}_{dataset['title']}.png", bbox_inches='tight')
        plt.close()

print("Plots generated for m=50, 200, 300. Check canvas for three images per m.")

Plots generated for m=50, 200, 300. Check canvas for three images per m.
